In [2]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import os

In [3]:

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["OPENAI_API_KEY"] = "sk-LKsUvDXZFn2KTTo371FVgqfXh6-RuldhQ7hvVzD-5sT3BlbkFJoxxPJsoPZEfU0ziK7zY-pjRgSXxH89aEq6xbWkxvUA"

In [4]:
def load_and_concatenate_files(file_paths=['RomanUrduDataSet.csv']):
    all_data = ""
    for file_path in file_paths:
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                all_data += file.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    return all_data


In [5]:
# extracted_data
extracted_data = load_and_concatenate_files()


In [6]:
# Function to split text into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100, chunk_overlap=20
    )
    text_chunks = text_splitter.split_text(extracted_data)
    return text_chunks


In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 22628


In [9]:
#Embedding Model
def OpenAIEmbeddings_Model():
   embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
   return embeddings


In [10]:
embeddings=OpenAIEmbeddings_Model()

In [11]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001C565BD77F0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001C565BD7430>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 3072


In [13]:
from langchain.schema import Document
# Convert texts to Document objects
documents = [Document(page_content=t) for t in text_chunks]

In [14]:
# Initialize vector database
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory='db'
)

In [15]:
vectordb = Chroma(persist_directory='db', embedding_function=embeddings)

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [17]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [34]:
# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Query: {query}\nJawab sirf Roman Urdu mein dein."
)

In [40]:
from langchain import PromptTemplate

# Ensure prompt_template is a string
prompt_template = "Provide an answer based on the context: {context} and the question: {question}"

# Initialize PromptTemplate correctly
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}


In [41]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [42]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        # Check if 'source' key exists before accessing it
        if 'source' in source.metadata:
            print(source.metadata['source'])
        else:
            print("Source metadata does not contain 'source' key.")

In [45]:
result = qa({"query": "Dunya ma kitna Mulk hain?"})
process_llm_response(result)



Dunya ma taqreeban 195 mulk hain, lekin is ki tadad badhti ja rahi hai. Har mulk apni khudai hakumat mai hota hai aur apni tareekh, maishat, aur saqafat rakhta hai. Har mulk apne tareekhi, maishati, aur siyasi imkanat ke sath mukhtalif hai.


Sources:
Source metadata does not contain 'source' key.
Source metadata does not contain 'source' key.


In [36]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}